In [1]:
# 셀 1: 필수 패키지 설치
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 

In [2]:
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 옵션
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 크롤링 대상 URL 리스트
urls = [
    "https://www.lge.co.kr/category/wash-tower",
    "https://www.lge.co.kr/category/wash-combo",
    "https://www.lge.co.kr/category/washing-machines",
    "https://www.lge.co.kr/category/dryers"
]

def scroll_through_all_lis(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        lis = driver.find_elements("css selector", "section.CommonPcListUnitProduct_list_unit_product__AeuOT ul[role='list'] > li")
        for li in lis:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", li)
                time.sleep(0.3)
            except:
                continue
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def scrape_products_with_links(url):
    driver.get(url)
    time.sleep(3)
    scroll_through_all_lis(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    section = soup.find('section', class_="CommonPcListUnitProduct_list_unit_product__AeuOT")
    products = []
    base_url = get_base_url(url)
    if section:
        ul = section.find('ul', role='list')
        if ul:
            lis = ul.find_all('li')
            for li in lis:
                data = li.get('data-ec-product')
                a_tag = li.find('a', href=True)
                link = a_tag['href'] if a_tag else None
                if data and link:
                    products.append({
                        'ec_product': data,
                        'detail_link': base_url + link if link.startswith('/') else base_url + '/' + link
                    })
    return products

def get_base_url(url):
    idx = url.find('/category')
    if idx != -1:
        return url[:idx]
    else:
        return url

BASE_URL = "https://www.lge.co.kr"

def get_soup(driver, url):
    driver.get(url)
    time.sleep(2)
    return BeautifulSoup(driver.page_source, 'html.parser')

def make_absolute_url(src, base_url=BASE_URL):
    if src.startswith("http"):
        return src
    return base_url + src

def extract_images(soup):
    images = []
    div = soup.find('div', class_="slide-content pdp-visual ui_carousel_list ui_static draggable")
    if div:
        ul = div.find('ul', class_="slide-track pdp-visual-list ui_carousel_track ui_static")
        if ul:
            for li in ul.find_all('li'):
                li_class = ' '.join(li.get('class', []))
                if (li_class == "slide-conts ui_carousel_slide default thumbnail img_square" or
                    li_class == "slide-conts ui_carousel_slide default thumbnail img_square ui_carousel_current on"):
                    a_tag = li.find('a')
                    if a_tag:
                        img_tag = a_tag.find('img')
                        if img_tag and img_tag.has_attr('src'):
                            img_src = make_absolute_url(img_tag['src'])
                            images.append({'img_src': img_src})
    return images

def extract_specs(soup):
    specs = []
    prod_spec_div = soup.find('div', class_='prod-spec-detail')
    if prod_spec_div:
        box_div = prod_spec_div.find('div', class_='box')
        spec_title = box_div.find('h3', class_='tit').get_text(strip=True) if box_div and box_div.find('h3', class_='tit') else ''
        spec_info_div = prod_spec_div.find('div', class_='spec-info-list')
        if spec_info_div:
            ul_tag = spec_info_div.find('ul')
            if ul_tag:
                for li in ul_tag.find_all('li'):
                    dl_tag = li.find('dl')
                    if dl_tag:
                        dt_tag = dl_tag.find('dt')
                        dd_tag = dl_tag.find('dd')
                        dt_text = ''
                        dt_btn_data = None
                        if dt_tag:
                            button_tag = dt_tag.find('button')
                            if button_tag and button_tag.has_attr('data-spec-description'):
                                dt_btn_data = button_tag['data-spec-description']
                            dt_text = dt_tag.get_text(strip=True)
                        dd_text = dd_tag.get_text(strip=True) if dd_tag else ''
                        specs.append({
                            'spec_title': spec_title,
                            'dt': dt_text,
                            'dd': dd_text,
                            'dt_button_data_spec_description': dt_btn_data
                        })
    return specs

def get_product_detail_info(driver, detail_url):
    soup = get_soup(driver, detail_url)
    return {
        'images': extract_images(soup),
        'specs': extract_specs(soup)
    }

# --- 통합 데이터 수집 및 매핑 ---
all_products = {}

for url in urls:
    print(f"\n크롤링 중: {url}")

    # 상품 데이터 수집
    products = scrape_products_with_links(url)
    product_list = []
    for prod in products:
        try:
            item = json.loads(prod['ec_product'])
            sku = item.get('model_sku', '').replace('.AKOR', '').strip()
            detail_url = prod['detail_link']
            try:
                detail_info = get_product_detail_info(driver, detail_url)
            except Exception as e:
                print(f"상세페이지 크롤링 실패: {detail_url} ({e})")
                detail_info = {'images': [], 'specs': []}
            product_list.append({
                'model_name': item.get('model_name', ''),
                'model_sku': sku,
                'price': item.get('price', ''),
                'brand': item.get('brand', ''),
                'detail_url': detail_url,
                'images': detail_info['images'],
                'specs': detail_info['specs']
            })
        except Exception as e:
            print(f"상품 정보 파싱 실패: {prod} ({e})")
            continue

    all_products[url] = product_list

# --- 결과 출력 ---
for category_url, products in all_products.items():
    print(f"\n=== 카테고리: {category_url}")
    for prod in products:
        print(f"\n상품명: {prod['model_name']}")
        print(f"모델명: {prod['model_sku']}")
        print(f"가격: {prod['price']}")
        print(f"브랜드: {prod['brand']}")
        print("이미지:")
        for img in prod['images']:
            print(f"  - {img['img_src']}")
        print("스펙:")
        for spec in prod['specs']:
            print(f"  - [{spec['dt']}] {spec['dd']} (details: {spec['dt_button_data_spec_description']})")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1-2인
· 트롬 : 12~15kg
· 통돌이 세탁기: 10~12kg

1인 가구 일주일 치 세탁물이 4kg임을 감안해 10kg 초반 작은 용량을 추천드립니다.

2-3인
· 트롬 : 21~23kg
· 통돌이 세탁기: 15~16kg

겉옷이나 속옷을 나눠 세탁하거나 땀 흘린 운동복, 반려동물 장난감 등 별도 세탁물이 많다면 기존 트롬에 미니워시를 추가해도 좋습니다.

3인 이상
· 트롬 : 21~25kg
· 통돌이 세탁기: 20~24kg
· 트롬 트윈워시: 28~29kg

드럼 방식은 세탁시 낙차를 이용한 여유 공간이 필요해 통돌이 세탁기보다 세탁 용량을 20~30% 크게 잡으면 좋습니다.)

상품명: LG 트롬 오브제컬렉션 + 건조기 + 스태킹키트
모델명: KFG19EN-8EN
가격: 2,278,500
브랜드: LG
이미지:
  - https://www.lge.co.kr/kr/images/washing-machines/md10066844/gallery/medium02.jpg
  - https://www.lge.co.kr/kr/images/washing-machines/md10066844/gallery/medium01.jpg
스펙:
  - [세탁 용량 (kg)] 19 (details: 1인 기준 일주일치 빨랫감은 평균 4~5kg* 으로 구성원 수와 사용패턴을 고려하여 용량 선택에 참고 하실 수 있습니다.

[가구 유형별 추천 용량]

1-2인
· 트롬 : 12~15kg
· 통돌이 세탁기: 10~12kg

1인 가구 일주일 치 세탁물이 4kg임을 감안해 10kg 초반 작은 용량을 추천드립니다.

2-3인
· 트롬 : 21~23kg
· 통돌이 세탁기: 15~16kg

겉옷이나 속옷을 나눠 세탁하거나 땀 흘린 운동복, 반려동물 장난감 등 별도 세탁물이 많다면 기존 트롬에 미니워시를 추가해도 좋습니다.

3인 이상
· 트롬 : 21~25kg
· 통돌이 세탁기: 20~24kg
· 트롬 트윈워시: 28~29

In [3]:
# --- 결과 저장 ---
import pandas as pd

all_rows = []
for category_url, products in all_products.items():
    for prod in products:
        row = prod.copy()
        row['category_url'] = category_url
        row['images'] = json.dumps(prod['images'], ensure_ascii=False)
        row['specs'] = json.dumps(prod['specs'], ensure_ascii=False)
        all_rows.append(row)
df = pd.DataFrame(all_rows)
df.to_csv('lge_products.csv', index=False)